In [54]:
import datetime

# Main entry point for DataFrame and SQL functionality.
from pyspark.sql import SparkSession

In [55]:
# Start SPARK Session
spark = SparkSession.builder.appName('Basics').getOrCreate()

In [56]:
# import pandas as pd

# # import data
# data = pd.read_csv("/home/bluepi/Downloads/Update/product_info/main table/product_info.csv")

# # convert day column type to datetime
# data['day'] = pd.to_datetime( data['day'], infer_datetime_format=True, yearfirst=True)

# # create new column
# data['date_timestamp'] = pd.to_datetime(data.day.astype(str) + ' ' + data.time)

# # Drop old columns
# data.drop(['day','time'],inplace=True,axis=1)

# # Write to csv
# data.to_csv("/home/bluepi/Downloads/Update/Updated Product/Latest Product/main table.csv", index=False)

In [57]:
mainTable = spark.read.format('csv').options(
    header=True, inferschema=True).load(
        "/home/bluepi/Downloads/Update/product_info/main table/main table.csv")

In [58]:
mainTable.printSchema()

root
 |-- p_id: integer (nullable = true)
 |-- p_name: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- date_timestamp: timestamp (nullable = true)



In [59]:
from pyspark.sql.functions import year, month, dayofyear, hour, minute, second

mainTable.select([year(mainTable['date_timestamp']).alias("Year"),
                  month(mainTable['date_timestamp']).alias("Month"),
                  dayofyear(mainTable['date_timestamp']).alias("DayOfYear"),
                  hour(mainTable['date_timestamp']).alias("Hour"),
                  minute(mainTable['date_timestamp']).alias("Minute"),
                  second(mainTable['date_timestamp']).alias("Second")]).show(3)

+----+-----+---------+----+------+------+
|Year|Month|DayOfYear|Hour|Minute|Second|
+----+-----+---------+----+------+------+
|2020|    5|      123|   8|     6|    42|
|2020|    2|       33|   8|     2|    22|
|2020|    2|       59|   7|    58|     8|
+----+-----+---------+----+------+------+
only showing top 3 rows



In [60]:
mainTable.select('date_timestamp').orderBy(mainTable.date_timestamp.asc()).show(6)

+-------------------+
|     date_timestamp|
+-------------------+
|2020-01-02 01:25:16|
|2020-01-02 05:05:59|
|2020-01-02 05:54:01|
|2020-01-02 09:08:05|
|2020-01-02 20:11:17|
|2020-02-02 01:16:34|
+-------------------+
only showing top 6 rows



In [61]:
# Address to the product_info folder
address = "/home/bluepi/Downloads/Update/product_info/"
previous_day = (datetime.datetime.today() - datetime.timedelta(days=1)).strftime('%d-%m-%Y')

# Address to the Previous Day folder
new_address = address + previous_day

# Read the Previous Day folder
per_day_data = spark.read.format('csv') \
          .options( header=True, inferschema=True ) \
          .load(new_address)

In [62]:
# per_day_data.show(3)
per_day_data.orderBy(per_day_data.Date_timestamp.asc()).show(60)
# per_day_data.printSchema()

+----+-----------+-----+-------------------+-----------+
|p_id|     p_name|price|     Date_timestamp|record_type|
+----+-----------+-----+-------------------+-----------+
| 195|    Redhold|  734|2020-04-05 03:19:16|          U|
|  68|   Alphazap|  442|2020-04-05 03:20:39|          U|
| 180|Solarbreeze|  521|2020-04-05 03:27:13|          U|
|  34|   Home Ing| 1063|2020-04-05 03:35:58|          U|
| 187|        Tin|  919|2020-04-05 03:41:04|          U|
|  50|     Bamity|  990|2020-04-05 09:10:40|          U|
| 178|   Home Ing| 1481|2020-04-05 09:14:33|          U|
|  79|   Zaam-Dox|  701|2020-04-05 09:31:11|          D|
|  67|    Fintone|  121|2020-04-05 09:32:19|          U|
|  37|  Ronstring|  462|2020-04-05 09:48:13|          U|
| 138|  Lotstring|  694|2020-04-05 09:49:46|          U|
|  31|    Matsoft|  122|2020-04-05 09:58:34|          U|
|  59|    Matsoft|  565|2020-04-05 15:13:05|          U|
|  35|     Lotlux|  831|2020-04-05 15:17:50|          U|
| 218|      Subin|  856|2020-04

In [10]:
# # Create the schema
# from pyspark.sql.types import *

# schema = StructType([StructField("p_id", IntegerType(), True),
#                      StructField("p_name", StringType(), True),
#                      StructField("price", IntegerType(), True),
#                      StructField("Date_timestamp", TimestampType() , True),
#                      StructField("record_type", StringType(), True)
#                      ])
# # Create latest proct table
# # Initially it is empty no updates
# Latest_Product_Table = spark.createDataFrame([], schema)

In [11]:
# Directly append new Inserted products
from pyspark.sql.functions import *
per_day_data_1 = per_day_data.filter(
    per_day_data.record_type == 'U').select(['p_id', 'date_timestamp'])

ppd_1 = per_day_data_1.alias('ppd_1')
ppd_2 = per_day_data_1.alias('ppd_2')

per_day_data_2 = ppd_1.join(
    ppd_2, ppd_1.p_id == ppd_2.p_id, 'inner').where(ppd_1.date_timestamp > ppd_2.date_timestamp)

per_day_data_2.show()
# per_day_data_2 = per_day_data_1.select(
#     ['p_id', 'p_name', 'price', 'Date_timestamp'])
# mainTable_1 = mainTable.union(per_day_data_2)

# # mainTable_1.count()

+----+--------------+----+--------------+
|p_id|date_timestamp|p_id|date_timestamp|
+----+--------------+----+--------------+
+----+--------------+----+--------------+



In [12]:
ppd_2.select("hour("date_add_timestamp").select()

SyntaxError: invalid syntax (<ipython-input-12-3a1708ccd6af>, line 1)

In [13]:
# Drop deleted products

# Registers this DataFrame as a temporary table using the given name
mainTable_1.registerTempTable("table_main")
per_day_data.registerTempTable("table_day")

mainTable_2 = spark.sql(
    "select * from table_main where p_id not in ( select p_id from table_day where record_type = 'D' )")

# mainTable_2.count()

NameError: name 'mainTable_1' is not defined

In [14]:
# update products
from pyspark.sql.functions import *
mainTable_2.select( countDistinct(mainTable_2.p_id) ).show()

NameError: name 'mainTable_2' is not defined

In [ ]:
mainTable_2.select('p_id').groupBy("p_id").count().filter( "count > 1" ).show()